# This notebook is intended to get the current AOS data for a given exposure.

Craig Lage - 05-Nov-24

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.ts.xml.tables.m1m3 import FATable
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
%matplotlib inline

In [ ]:
#butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"]) # USDF
butler = Butler('/repo/LSSTComCam', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"]) #Summit
client = makeEfdClient()

In [ ]:
axials = np.array([[0, 1.601],
               [0.33287, 1.56601],
               [0.65119, 1.46259],
               [0.94104, 1.29524],
               [1.18977, 1.07128],
               [1.38651, 0.8005],
               [1.52264, 0.49474],
               [1.59223, 0.16735],
               [1.59223, -0.16735],
               [1.52264, -0.49474],
               [1.38651, -0.8005],
               [1.18977, -1.07128],
               [0.94104, -1.29524],
               [0.65119, -1.46259],
               [0.33287, -1.56601],
               [0, -1.601],
               [-0.33287, -1.56601],
               [-0.65119, -1.46259],
               [-0.94104, -1.29524],
               [-1.18977, -1.07128],
               [-1.38651, -0.8005],
               [-1.52264, -0.49474],
               [-1.59223, -0.16735],
               [-1.59223, 0.16735],
               [-1.52264, 0.49474],
               [-1.38651, 0.8005],
               [-1.18977, 1.07128],
               [-0.94104, 1.29524],
               [-0.65119, 1.46259],
               [-0.33287, 1.56601],
               [0.1676, 1.27302],
               [0.49137, 1.18626],
               [0.78165, 1.01867],
               [1.01867, 0.78165],
               [1.18626, 0.49137],
               [1.27302, 0.1676],
               [1.27302, -0.1676],
               [1.18626, -0.49137],
               [1.01867, -0.78165],
               [0.78165, -1.01867],
               [0.49137, -1.18626],
               [0.1676, -1.27302],
               [-0.1676, -1.27302],
               [-0.49137, -1.18626],
               [-0.78165, -1.01867],
               [-1.01867, -0.78165],
               [-1.18626, -0.49137],
               [-1.27302, -0.1676 ],
               [-1.27302, 0.1676 ],
               [-1.18626, 0.49137],
               [-1.01867, 0.78165],
               [-0.78165, 1.01867],
               [-0.49137, 1.18626],
               [-0.1676, 1.27302],
               [0, 1.002],
               [0.3427, 0.94157],
               [0.64407, 0.76758],
               [0.86776, 0.501],
               [0.98678, 0.174],
               [0.98678, -0.174],
               [0.86776, -0.501],
               [0.64407, -0.76758],
               [0.3427, -0.94157],
               [0, -1.002],
               [-0.3427, -0.94157],
               [-0.64407, -0.76758],
               [-0.86776, -0.501],
               [-0.98678, -0.174],
               [-0.98678, 0.174],
               [-0.86776, 0.501],
               [-0.64407, 0.76758],
               [-0.3427, 0.94157]])


In [ ]:
# Define the names of the degrees of freedom
DOF_names = []
for component in ["M2", "Cam"]:
    for dof in ["dz", "dx", "dy", "rx", "ry"]:
        DOF_names.append(component+"_"+dof)
for i in range(20):
    DOF_names.append(f"M1M3_B{i+1}")
for i in range(20):
    DOF_names.append(f"M2_B{i+1}")

In [ ]:
def plotM1M3_AOS(df, ax, FATable, zmin=-200, zmax=200):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-4.5,4.5)
    ax.set_title("M1M3 AOS forces (N)", fontsize=12)
    if len(df) == 0:
        ax.text(-2.0, 0, "Not Available")
        return

    index = -1
    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"zForces{i}"
                zs.append(df.iloc[index][name])
        im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', \
                    norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                     s=50, label=label)
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r') 
    return

def plotM2_AOS(df, ax, axials, zmin=-200, zmax=200):
    # Get the data from the yaml file
    scale = 2.5
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M2 AOS forces (N)", fontsize=12)

    if len(df) == 0:
        ax.text(-1, 0, "Not Available")
        return

    index = -1
    xs = axials[:,0]
    ys = axials[:,1]
    zs = []
    for i in range(len(xs)):
        name=f"axial{i}"
        force = df.iloc[index][name]
        zs.append(force)

    im = ax.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', \
                    norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                     s=80, label="Axial")
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.0, cmap='RdBu_r')
    return


def getData(client, expRecord, fig):
    # Set up the axes
    ax1 = fig.add_axes([0.10,0.45,0.80,0.45])
    ax2 = fig.add_axes([0.20,0.10,0.24,0.30])
    ax3 = fig.add_axes([0.60,0.10,0.24,0.30])
    ax1.set_xticks([])
    ax1.set_yticks([])
    # Basic text
    plt.suptitle(f"{expId}", fontsize=18)
    tOpen = Time(expRecord.timespan.begin, scale='tai').utc
    tClose = Time(expRecord.timespan.end, scale='tai').utc
    text1 = f"Topen = {tOpen.isot}, Tclose = {tClose.isot}"
    ax1.text(0.1, 0.95, text1, color='black')
    position = f"Azimuth = {expRecord.azimuth:.2f}, "
    position += f"Elevation = {(90.0 - expRecord.zenith_angle):.2f}, "
    position += f"Rotation = {expRecord.sky_angle:.2f}"
    ax1.text(0.1, 0.87, position, color='black')
    # Hexapod status    
    hexData = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        expRecord=expRecord
    )
    camHex = hexData[hexData['salIndex'] == 1]
    m2Hex = hexData[hexData['salIndex'] == 2]
    names = ['Camera', 'M2']
    yText = [0.79, 0.71]
    for i, hex in enumerate([camHex, m2Hex]):
        textHex = f"{names[i]} hexapod: "
        X = hex.iloc[0]["position0"]
        Y = hex.iloc[0]["position1"]
        Z = hex.iloc[0]["position2"]
        U = hex.iloc[0]["position3"]
        V = hex.iloc[0]["position4"]
        hexPos = f"X={X:.1f}um, Y={Y:.1f}um, Z={Z:.1f}um, U = {U * 3600.0:.1f} arcsec, V = {V * 3600.0:.1f} arcsec"
        print(names[i], hexPos)
        textHex += hexPos  
        ax1.text(0.1, yText[i], textHex, color='black')
    # AOS DOF status
    offsetDOF = getEfdData(
        client,
        "lsst.sal.MTAOS.logevent_degreeOfFreedom",
        expRecord=expRecord,
        prePadding=7200
    )
    
    textBends = [""]
    counter = 0
    textBends[counter] += "AOS modes: "
    nModes = 0
    for i in range(50):
        value = offsetDOF.iloc[-1][f"aggregatedDoF{i}"]
        if abs(value) > 1.0E-6:
            textBends[counter] += f"{DOF_names[i]} = {value:.2f}, "
            nModes += 1
            if nModes in [5, 11, 17, 23, 29]:
                textBends.append("")
                counter += 1
    for ii in range(counter + 1):
        print(textBends[ii])
        ax1.text(0.05, 0.63-0.08*ii, textBends[ii], color='black', clip_on=False)

    M1M3_AOS_names = []
    for i in range(156):
        name=f"zForces{i}"
        M1M3_AOS_names.append(name)
    
    M2_AOS_names = []
    for i in range(72):
        name=f"axial{i}"
        M2_AOS_names.append(name)

    M1M3_AOS = getEfdData(
        client,
        "lsst.sal.MTM1M3.command_applyActiveOpticForces",
        expRecord=expRecord,
        prePadding=7200,
        columns=M1M3_AOS_names,
    )

    plotM1M3_AOS(M1M3_AOS, ax2, FATable, zmin=-200, zmax=200)

    M2_AOS = getEfdData(
        client,
        "lsst.sal.MTM2.command_applyForces",
        expRecord=expRecord,
        prePadding=7200,
        columns=M2_AOS_names,
    )
    
    plotM2_AOS(M2_AOS, ax3, axials, zmin=-200, zmax=200)
    return fig


In [ ]:
fig = plt.figure(figsize=(10,8))
expId = 2024110600252
print(expId)
dataId = {'exposure': expId, 'detector': 4, 'instrument': 'LSSTComCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
fig = getData(client, expRecord, fig)
plt.savefig(f"/home/cslage/MTAOS/images/ComCam_DOFs_{expId}_07Nov24.png")

In [ ]:
Cam_dx: -141.85
Cam_dy: 673.96
Cam_dz: 1972.47
Cam_rx: 7.76e-3
Cam_ry: 0
M2_dz: -1265.88

AOS modes: M2_dz = -1265.88, Cam_dz = 1972.47, Cam_dx = -141.85, Cam_dy = 673.96, Cam_rx = 0.01, 

In [ ]:
2024110500233
Camera X=4858.1um, Y=535.3um, Z=2199.2um, U = 13.9 arcsec, V = -0.1 arcsec
M2 X=-2.9um, Y=-117.4um, Z=-1014.9um, U = -5.4 arcsec, V = -0.0 arcsec
AOS modes: M2_dz = -1265.88, Cam_dz = 1972.47, Cam_dx = 4858.15, Cam_dy = 673.96, Cam_rx = 0.01, 

2024110500234
Camera X=4856.6um, Y=548.3um, Z=2195.6um, U = 15.1 arcsec, V = 0.1 arcsec
M2 X=-3.0um, Y=-106.1um, Z=-1018.4um, U = -4.9 arcsec, V = -0.0 arcsec
AOS modes: M2_dz = -1265.88, Cam_dz = 1972.47, Cam_dx = 4858.15, Cam_dy = 673.96, Cam_rx = 0.01, 